In [31]:
import torch
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from torchvision import transforms
import numpy as np

In [26]:
class CityDataset(Dataset):
    def __init__(self, list_file):
        super(CityDataset, self).__init__()

        self.path_list = []
        with open(list_file, "r") as fin:
            for line in fin.readlines():
                self.path_list.append(line.strip())
        self.preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    def __len__(self):
        return len(self.path_list)

    def __getitem__(self, index):
        img = Image.open(self.path_list[index]).convert("RGB")
        img = self.preprocess(img)
        return img
    
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()
model.to('cuda')

Using cache found in C:\Users\Administrator/.cache\torch\hub\pytorch_vision_v0.10.0


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [33]:
d = CityDataset("list.txt")
d_loader = DataLoader(d, batch_size=12)

batch_n = len(d)/12

with torch.no_grad():
    for idx, x in enumerate(d_loader):
        x = x.cuda()
        y = model(x)['out'].argmax(1).cpu().numpy()
#         print(y.shape)
        np.save(f"deeplabv3_predictions/bs12_batch{idx:04d}.npy", y)
        if idx % 10 == 0:
            print(f"Batch {idx}/{batch_n}")
print("DONE.")

(12, 480, 853)


86124

In [29]:
f"{1:04d}"

'0001'